In [ ]:
import os
import random
from glob import glob

In [29]:
base_path = "../data/PetImages"

cats = glob(os.path.join(base_path, "Cat", "*.jpg"))
dogs = glob(os.path.join(base_path, "Dog", "*.jpg"))

print("Jumlah Gambar Kucing: ", len(cats))
print("Jumlah Gambar Anjing: ", len(dogs))

Jumlah Gambar Kucing:  12500
Jumlah Gambar Anjing:  12500


In [33]:
def split_dataset(cats, dogs, val_ratio=0.2):
    random.shuffle(cats)
    random.shuffle(dogs)

    n_cats_val = int(len(cats) * val_ratio)
    n_dogs_val = int(len(dogs) * val_ratio)

    n_val = cats[n_cats_val:] + dogs[n_dogs_val:]
    n_train = cats[:n_cats_val] + dogs[:n_dogs_val]

    return n_val, n_train

n_train, n_val = split_dataset(cats, dogs)
print("Jumlah Data Latih: ", len(n_train))
print("Jumlag Data Validasi: ", len(n_val))

Jumlah Data Latih:  20000
Jumlag Data Validasi:  5000


In [50]:
def move_files(n_data, direktori_baru):
    os.makedirs(os.path.join(direktori_baru, "Cat"), exist_ok=True)
    os.makedirs(os.path.join(direktori_baru, "Dog"), exist_ok=True)

    for f in n_data:
        label = "Cat" if "Cat" in f else "Dog"
        shutil.copy(f, os.path.join(direktori_baru, label, os.path.basename(f)))

move_files(n_train, "../data/PetImages/data_train")
move_files(n_val, "../data/PetImages/data_val")
